Cargamos las imagenes almacenadas en la carpeta "images" con el codigo DataSetReduced y se redimensionan segun img_dim

In [1]:
import matplotlib.pyplot as plt
import csv
import scipy
from scipy import misc
import numpy as np

################ CONSTANTES #################
#### modificar en caso que sea necesario ####
ruta = './images/' # ruta donde se almacenaron las imagenes creadas con el DataSetReduced.py
img_dim= (128, 128) # tamaño de las imagenes redimensionadas

# solo es util en caso de que se ejecute el codigo más de 1 vez
try:del X
except: None
try:del Y
except: None

with open(ruta + 'info.csv', 'r') as info: # informacion de las imagenes
    reader = csv.reader(info) # csv parser for annotations file
    # lee cada imagen con su clasificacion contenida en el archivo de informacion
    for row in reader:
        img = plt.imread(ruta + row[0]) # la primera columna es el nombre del archivo
        res_img= scipy.misc.imresize(img, img_dim) #redimensiono la imagen
        res_img = np.expand_dims(res_img, axis=0) #agrego una dimension para poder aplicar una concatenacion
        
        #si no esta declarado el arreglo lo crea
        try: X = np.append(X, res_img, axis=0)
        except: X = np.array(res_img)
            
        try: Y = np.append(Y, row[1]) # la segunda columna es la clasificacion
        except: Y = np.array(row[1])
        
        

Se reducen los canales a 1.

Keras no tiene una funcion que convierta una imagen RGB en escala de grises. Por lo que se utilizara la libreria numpy para realizar este trabajo.

!!Dejar comentado este codigo si se desea utilizar 3 canales

In [ ]:
#convierte la imagen en escala de grises
#try:
#    plt.imshow(X[1])
#    plt.show()
#except: print("los valores originales de X ya no estan disponibles :C")
#else:
#    X = np.sum(X/3, axis=3, keepdims=True)
##squeeze elimina las dimensiones de tamaño 1
##ya que matplotlib exige que el canal sea 3 o 4
##o que la matriz tenga 2 dimensiones
#plt.imshow(np.squeeze(X[1]), cmap='gray')
#plt.show()

Lo siguiente será mezclar las entradas y tomar una parte para el train y otra para el validation

In [2]:
import random
XYtuple = []
for i in range(len(X)):
    XYtuple.append((X[i], Y[i]))
    
xx= []
yy= []

XYtuple = random.sample(XYtuple, len(XYtuple))

for i in range(len(XYtuple)):
    xx.append(XYtuple[i][0])
    yy.append(XYtuple[i][1])

X = np.array(xx)
Y = np.array(yy)

Se utilizará el 85% de los datos para entrenar la red, y el 15% para la validacion

In [3]:
#85% train 15%test    
train_len= (int)(len(X)*0.85)
X_train = X[:train_len]
Y_train = Y[:train_len]
X_valid = X[train_len:]
Y_valid = Y[train_len:]

modificamos la forma del arreglo para volverlo categorico
Y train es un arreglo unidimensional donde para cada imagen tiene un valor de 1 a 43 segun su clasificacion
Y train de salida es un arreglo bidimensional. Donde para cada imagen hay un arreglo de ancho 43 indicando su clasificacion en la posicion correspondiente

In [5]:
from keras.utils import np_utils

Y_train = np_utils.to_categorical(Y_train, 43)
print (Y_train.shape)
Y_valid = np_utils.to_categorical(Y_valid, 43)
print (Y_valid.shape)

Using TensorFlow backend.


(1110, 43)
(196, 43)


preprocesamiento de imagenes: reescalamiento, corrimiento vertical, corrimiento horizontal, rotacion y estiramiento

In [9]:
from keras.preprocessing.image import ImageDataGenerator

from keras import backend as K
from keras.callbacks import EarlyStopping

epochs = 20
batch_size = 16
batch_aug = 50
colors= 3
#earlystop= EarlyStopping(min_delta= 0.008, patience= 2, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', patience=2, min_delta = 0.005, verbose=1)
nb_train_samples = X_train.shape[0]
nb_validation_samples = X_valid.shape[0]


if K.image_data_format() == 'channels_first':
    input_shape = (colors, img_dim[0], img_dim[1])
else:
    input_shape = (img_dim[0], img_dim[1], colors)
                                      
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                    rotation_range=40,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    fill_mode='nearest')

# data augmentation:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow(X_train, Y_train, batch_size=30)

validation_generator = test_datagen.flow(X_valid, Y_valid, batch_size=30)


Found 1 images belonging to 43 classes.


Codigo de prueba para visualizar la transformacion de las imagenes

In [ ]:
#data augmentation testing

#import keras
#from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

#datagen = ImageDataGenerator(
#        rotation_range=40,
#        width_shift_range=0.1,
#        height_shift_range=0.1,
#        shear_range=0.2,
#        zoom_range=0.2,
#        fill_mode='nearest')

#x = X_train[1]
##x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
#x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
#print(x.shape)

## the .flow() command below generates batches of randomly transformed images
## and saves the results to the `augmentation/` directory
#i = 0
#for batch in datagen.flow(x, batch_size=1,
#                          save_to_dir='augmentation', 
#                          save_prefix='ts', 
#                          save_format='jpeg'):
#    i += 1
#    if i > 30:
#        break  # luego de 30 imagenes termina

Modelo de la red.

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), 
                 input_shape=input_shape, 
                 kernel_initializer='glorot_normal', 
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(43, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#model.fit(X_train, Y_train,
#            batch_size=batch_size, epochs=epochs,
#            verbose=1, validation_data=(X_valid, Y_valid))

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=(X_valid, Y_valid),
    validation_steps=nb_validation_samples // batch_size,
    callbacks = [earlystop])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
__________

Dicho modelo se fue alterando para realizar diferentes pruebas. Comenzando con una red sencilla de una única capa convolusional y una Fully Connected, y agregandole progresivamente nuevas capas. Los resultados obtenidos fueron muy malos, desde un 5% con la red sencilla y alcanzando un 60% utilizando imagenes en rgb y una red profunda. Debido a esto se decidio cambiar de estrategia y volver a utilizar el dataset completo, tanto de training como de test